In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import plotly.express as px
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from SleepClasses.AggregatedData import AggregatedData

# Mostra tutte le colonne
pd.set_option('display.max_columns', None)

In [2]:
sleep_dfs_agg = pd.read_pickle(r'SleepData/sleep_dfs_agg.pkl')
sleep_dfs_gran = pd.read_pickle(r'SleepData/sleep_dfs_gran.pkl')
agg = AggregatedData(sleep_dfs_gran['2'], sleep_dfs_agg['2'])

df = agg.getAggregatedDataDf()

/Users/boe/Desktop/Unimi/Tesi/code/SleepClasses/AggregatedData.py:575: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(merged_aggregated.values(), ignore_index=True)
/Users/boe/Desktop/Unimi/Tesi/code/SleepClasses/AggregatedData.py:628: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modified_rows_df = pd.concat([modified_rows_df, df.loc[[idx]]])
/Users/boe/Desktop/Unimi/Tesi/code/SleepClasses/AggregatedData.py:631: FutureWarning: The behavior of DataFrame concatenation with 

In [3]:
# Supponiamo che `df` sia il tuo DataFrame
measures = ["HrAverage", "HrMax", "HrMin", "RmssdAverage",
            "RmssdMax", "RmssdMin", "RrAverage", "RrMax", "RrMin", "Sdnn_1Average",
            "Sdnn_1Max", "Sdnn_1Min", "SnoringAverage", "SnoringMax", "SnoringMin",
            "LightSleepDuration", "RemSleepDuration", "DeepSleepDuration",
            "WakeUpDuration", "WakeUpCount", "ManualSleepDuration",
            "UndefinedSleepDuration", "OutOfBedCount", "OutOfBedTime",
            "TotalSleepTime"]

In [4]:
# Preprocessing: normalizzazione
scaler = MinMaxScaler()

# Normalizza il DataFrame
df[measures] = scaler.fit_transform(df[measures])

# Visualizza il DataFrame normalizzato
df

,Date,StartValue,EndValue,LightSleepDuration,DeepSleepDuration,RemSleepDuration,ManualSleepDuration,UndefinedSleepDuration,WakeUpDuration,TotalSleepTime,Nap,HrAverage,HrMax,HrMin,WakeUpCount,OutOfBedCount,model,model_id,RmssdAverage,RmssdMax,RmssdMin,RrAverage,RrMax,RrMin,Sdnn_1Average,Sdnn_1Max,Sdnn_1Min,SnoringAverage,SnoringMax,SnoringMin,OutOfBedTime,OutOfBed
0,2023-08-06,2023-08-05 21:42:00,2023-08-06 08:16:00,0.692157,1.000000,0.000000,0.0,0.0,0.138554,0.807428,False,0.000000,0.62,0.088889,0.500,0.000000,NaN,NaN,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,[]
1,2023-09-01,2023-09-01 14:17:00,2023-09-01 15:59:00,0.003922,0.140426,0.009217,0.0,0.0,0.195783,0.049519,True,0.750000,0.62,0.222222,0.125,0.000000,NaN,NaN,0.377358,0.196078,0.0,0.809524,0.806452,0.473684,0.314607,0.172897,0.0,0.000000,0.0,0.0,0.000000,[]
2,2023-09-01,2023-09-01 00:24:00,2023-09-01 08:32:00,0.549020,0.280851,0.087558,0.0,0.0,0.322289,0.500688,False,0.166667,0.36,0.111111,0.125,0.166667,NaN,NaN,0.603774,0.339869,0.0,0.714286,0.774194,0.473684,0.528090,0.257009,0.0,0.111111,1.0,0.0,0.114286,"[{'from_value': 1, 'to_value': 0, 'next_value'..."
3,2023-09-02,2023-09-02 16:14:00,2023-09-02 19:36:00,0.147059,0.153191,0.193548,0.0,0.0,0.132530,0.209078,True,0.138889,0.18,0.177778,0.125,0.000000,NaN,NaN,0.849057,0.947712,0.0,0.761905,0.741935,0.473684,0.550562,0.252336,0.0,0.000000,0.0,0.0,0.000000,[]
4,2023-09-02,2023-09-02 00:45:00,2023-09-02 08:05:00,0.517647,0.263830,0.433180,0.0,0.0,0.060241,0.576341,False,0.055556,0.12,0.088889,0.000,0.000000,NaN,NaN,0.622642,0.287582,0.0,0.714286,0.838710,0.631579,0.595506,0.299065,0.0,0.055556,1.0,0.0,0.000000,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,2024-11-10,2024-11-10 13:23:00,2024-11-10 14:20:00,0.015686,0.063830,0.000000,0.0,0.0,0.102410,0.030261,True,0.916667,0.68,0.933333,0.000,0.000000,32.0,63.0,0.471698,0.366013,0.0,0.904762,0.806452,0.578947,0.393258,0.794393,0.0,0.000000,0.0,0.0,0.000000,[]
651,2024-11-10,2024-11-10 00:39:00,2024-11-10 07:33:00,0.592157,0.242553,0.129032,0.0,0.0,0.081325,0.530949,False,0.222222,0.44,0.177778,0.125,0.166667,32.0,63.0,0.905660,0.555556,0.0,0.761905,0.709677,0.526316,0.651685,0.742991,0.0,0.583333,1.0,0.0,0.040000,"[{'from_value': 1, 'to_value': 0, 'next_value'..."
652,2024-11-11,2024-11-11 00:50:00,2024-11-11 08:16:00,0.443137,0.323404,0.207373,0.0,0.0,0.237952,0.475928,False,0.305556,0.40,0.222222,0.125,0.166667,32.0,63.0,0.849057,0.542484,0.0,0.761905,0.806452,0.526316,0.617978,0.962617,0.0,0.527778,1.0,0.0,0.131429,"[{'from_value': 3, 'to_value': 0, 'next_value'..."
653,2024-11-12,2024-11-11 23:08:00,2024-11-12 07:16:00,0.592157,0.157447,0.460829,0.0,0.0,0.147590,0.602476,False,0.194444,0.44,0.222222,0.125,0.166667,32.0,63.0,0.867925,0.529412,0.0,0.761905,0.774194,0.526316,0.573034,0.752336,0.0,0.166667,1.0,0.0,0.045714,"[{'from_value': 3, 'to_value': 0, 'next_value'..."


In [5]:
df['StartValue'] = pd.to_datetime(df['StartValue'])
df['EndValue'] = pd.to_datetime(df['EndValue'])

# Trova il primo giorno disponibile nel dataset
start_date = df['EndValue'].min()

# Calcola la differenza in giorni tra ogni data e la data iniziale
df['DaysSinceStart'] = (df['EndValue'] - start_date).dt.days

# Calcola il numero di settimane relative al primo giorno disponibile
df['WeekNumber'] = (df['DaysSinceStart'] // 7) + 1

# Aggiunta del primo e ultimo giorno della settimana come colonne
df['WeekStart'] = df.groupby('WeekNumber')['StartValue'].transform('min')
df['WeekEnd'] = df.groupby('WeekNumber')['EndValue'].transform('max')

# Raggruppa per settimana
weekly_data = df.groupby('WeekNumber').agg({
    'WeekStart':                'first',  # Usa il primo valore della settimana
    'WeekEnd':                  'first',    # Usa l'ultimo valore della settimana
    'HrAverage' :               ['mean', 'std', 'min', 'max'],
    'HrMax' :                   ['mean', 'std', 'min', 'max'],
    'HrMin' :                   ['mean', 'std', 'min', 'max'],
    'RmssdAverage' :            ['mean', 'std', 'min', 'max'],
    'RmssdMax' :                ['mean', 'std', 'min', 'max'],
    'RmssdMin' :                ['mean', 'std', 'min', 'max'],
    'RrAverage' :               ['mean', 'std', 'min', 'max'],
    'RrMax' :                   ['mean', 'std', 'min', 'max'],
    'RrMin' :                   ['mean', 'std', 'min', 'max'],
    'Sdnn_1Average' :           ['mean', 'std', 'min', 'max'],
    'Sdnn_1Max' :               ['mean', 'std', 'min', 'max'],
    'Sdnn_1Min' :               ['mean', 'std', 'min', 'max'],
    'SnoringAverage' :          ['mean', 'std', 'min', 'max'],
    'SnoringMax' :              ['mean', 'std', 'min', 'max'],
    'SnoringMin' :              ['mean', 'std', 'min', 'max'],
    'LightSleepDuration' :      ['mean', 'std', 'min', 'max'],
    'RemSleepDuration' :        ['mean', 'std', 'min', 'max'],
    'DeepSleepDuration' :       ['mean', 'std', 'min', 'max'],
    'WakeUpDuration' :          ['mean', 'std', 'min', 'max'],
    'WakeUpCount' :             ['mean', 'std', 'min', 'max'],
    'ManualSleepDuration' :     ['mean', 'std', 'min', 'max'],
    'UndefinedSleepDuration' :  ['mean', 'std', 'min', 'max'],
    'OutOfBedCount' :           ['mean', 'std', 'min', 'max'],
    'OutOfBedTime' :            ['mean', 'std', 'min', 'max'],
    'TotalSleepTime':           ['mean', 'std', 'min', 'max'],
    'Nap':                      'sum'  # Conta i pisolini in ogni settimana
}).reset_index()
columnList = ['WeekNumber',
                'WeekStart', 'WeekEnd',
                'MEAN_HrAverage', 'STD_HrAverage', 'MIN_HrAverage', 'MAX_HrAverage',
                'MEAN_HrMax', 'STD_HrMax', 'MIN_HrMax', 'MAX_HrMax',
                'MEAN_HrMin', 'STD_HrMin', 'MIN_HrMin', 'MAX_HrMin',
                'MEAN_RmssdAverage', 'STD_RmssdAverage', 'MIN_RmssdAverage', 'MAX_RmssdAverage',
                'MEAN_RmssdMax','STD_RmssdMax','MIN_RmssdMax','MAX_RmssdMax',
                'MEAN_RmssdMin','STD_RmssdMin','MIN_RmssdMin','MAX_RmssdMin',
                'MEAN_RrAverage','STD_RrAverage','MIN_RrAverage','MAX_RrAverage',
                'MEAN_RrMax','STD_RrMax','MIN_RrMax','MAX_RrMax',
                'MEAN_RrMin','STD_RrMin','MIN_RrMin','MAX_RrMin',
                'MEAN_Sdnn_1Average','STD_Sdnn_1Average','MIN_Sdnn_1Average','MAX_Sdnn_1Average',
                'MEAN_Sdnn_1Max','STD_Sdnn_1Max','MIN_Sdnn_1Max','MAX_Sdnn_1Max',
                'MEAN_Sdnn_1Min','STD_Sdnn_1Min','MIN_Sdnn_1Min','MAX_Sdnn_1Min',
                'MEAN_SnoringAverage','STD_SnoringAverage','MIN_SnoringAverage','MAX_SnoringAverage',
                'MEAN_SnoringMax','STD_SnoringMax','MIN_SnoringMax','MAX_SnoringMax',
                'MEAN_SnoringMin','STD_SnoringMin','MIN_SnoringMin','MAX_SnoringMin',
                'MEAN_LightSleepDuration','STD_LightSleepDuration','MIN_LightSleepDuration','MAX_LightSleepDuration',
                'MEAN_RemSleepDuration','STD_RemSleepDuration','MIN_RemSleepDuration','MAX_RemSleepDuration',
                'MEAN_DeepSleepDuration','STD_DeepSleepDuration','MIN_DeepSleepDuration','MAX_DeepSleepDuration',
                'MEAN_WakeUpDuration','STD_WakeUpDuration','MIN_WakeUpDuration','MAX_WakeUpDuration',
                'MEAN_WakeUpCount','STD_WakeUpCount','MIN_WakeUpCount','MAX_WakeUpCount',
                'MEAN_ManualSleepDuration','STD_ManualSleepDuration','MIN_ManualSleepDuration','MAX_ManualSleepDuration',
                'MEAN_UndefinedSleepDuration','STD_UndefinedSleepDuration','MIN_UndefinedSleepDuration','MAX_UndefinedSleepDuration',
                'MEAN_OutOfBedCount','STD_OutOfBedCount','MIN_OutOfBedCount','MAX_OutOfBedCount',
                'MEAN_OutOfBedTime','STD_OutOfBedTime','MIN_OutOfBedTime','MAX_OutOfBedTime',
                'MEAN_TotalSleepTime', 'STD_TotalSleepTime', 'MIN_TotalSleepTime', 'MAX_TotalSleepTime',
                'SUM_Nap']
# Rinomina colonne per facilità di utilizzo
weekly_data.columns = columnList

# Aggiunta di una colonna con il range di date della settimana
weekly_data['DateRange'] = weekly_data['WeekStart'].dt.strftime('%d %b %Y') + ' - ' + weekly_data['WeekEnd'].dt.strftime('%d %b %Y')

# Imposta la colonna 'std' a 0 nei casi in cui abbia NaN
columns_with_std = [col for col in weekly_data.columns if 'std' in col.lower()]  # Trova tutte le colonne 'std'

#ci sono casi in cui la standard deviation non si pò fare perchè la persona ha dormito solo 1 volta nella settimana, quindi metto a 0 la std
for col in columns_with_std:
    weekly_data[col] = weekly_data[col].fillna(0)  # Riempie i NaN solo nelle colonne 'std's

In [6]:
columnsToModel = [col for col in columnList if col not in ['WeekNumber','WeekStart', 'WeekEnd', 'DateRange', 'SUM_Nap']]
df_model = weekly_data[columnsToModel]
df_model = df_model.astype('float64')

In [7]:
# Reshape per avere una sequenza temporale di lunghezza 1 per ogni settimana
X = df_model.values

In [8]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers, output_dim):
        super(TransformerModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.num_layers = num_layers

        # Encoder del Transformer
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer, num_layers=num_layers)
        
        # Layer finale per ottenere la stessa dimensione delle feature in input
        self.fc = nn.Linear(input_dim, output_dim)  # output_dim dovrebbe essere 101

    def forward(self, x):
        # Elabora il batch (batch_size, sequence_length, input_dim)
        x = self.transformer_encoder(x)  # (batch_size, sequence_length, input_dim)

        # Rimuovi la dimensione della sequenza (sequence_length è 1 per una settimana)
        x = x.squeeze(1)  # La forma diventa (batch_size, input_dim)
        
        # Passa attraverso il layer finale per ottenere l'output corretto
        x = self.fc(x)  # output dovrebbe avere la forma (batch_size, 101)
        
        return x

In [9]:
# Converti in tensor
X_tensor = torch.tensor(X, dtype=torch.float32)

# Crea un dataset e un DataLoader
dataset = TensorDataset(X_tensor)
batch_size = 32  # Puoi scegliere il batch_size in base alla tua memoria

# Crea il DataLoader
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [10]:
# Imposta i parametri del modello
input_dim = 100  # Le tue feature settimanali
hidden_dim = 64
num_heads = 4
num_layers = 2
output_dim = 1  # Output per la classificazione delle anomalie

# Crea il modello
model = TransformerModel(input_dim, hidden_dim, num_heads, num_layers, output_dim)

In [11]:
criterion = nn.MSELoss()  # MSE per la ricostruzione
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam come ottimizzatore

In [12]:
# 4. Addestramento del modello
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Numero di epoche
num_epochs = 100

# Addestramento
for epoch in range(num_epochs):
    model.train()
    for X_batch in data_loader:
        X_batch = X_batch[0]  # Prendi il batch di input (ignoriamo il target, in quanto è non supervisionato)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, X_batch)  # La perdita è calcolata confrontando l'output con l'input
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")


/Users/boe/Desktop/Unimi/Tesi/.venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 100])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/boe/Desktop/Unimi/Tesi/.venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([28, 100])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 0.1022
Epoch [20/100], Loss: 0.0886
Epoch [30/100], Loss: 0.0869
Epoch [40/100], Loss: 0.0889
Epoch [50/100], Loss: 0.0888
Epoch [60/100], Loss: 0.0871
Epoch [70/100], Loss: 0.0840
Epoch [80/100], Loss: 0.0824
Epoch [90/100], Loss: 0.0821
Epoch [100/100], Loss: 0.0831


In [13]:
# 1. Otteniamo gli embeddings dal modello Transformer (senza classificare come anomali o normali)
model.eval()
embeddings = []

with torch.no_grad():
    for X_batch in X_tensor:
        # Ottieni gli embeddings (output prima della classificazione finale)
        embedding = model.encoder_layer(X_batch.unsqueeze(0))
        embeddings.append(embedding.squeeze().numpy())

# 2. Riduci la dimensionalità usando PCA per ottenere una rappresentazione 2D
pca = PCA(n_components=1)
embeddings = pca.fit_transform(embeddings)

# 3. Crea un DataFrame con gli embeddings ridotti
df_embeddings = pd.DataFrame(embeddings, columns=['Value'])
df_embeddings['Settimana'] = np.arange(1, len(df_embeddings) + 1)

In [14]:
# 4. Visualizza il grafico 2D delle settimane
fig = px.scatter(df_embeddings, x='Settimana', y='Value', title="Andamento della Dormita (PCA Embeddings)",
                 labels={'Settimana': 'Settimana', 'Value': 'Embedding Value'})

fig.show()

In [15]:
# 5. Calcola l'errore di ricostruzione
reconstruction_errors = []

model.eval()
with torch.no_grad():
    for X_batch in data_loader:
        X_batch = X_batch[0]
        reconstructed = model(X_batch)  # Il modello ricostruisce i dati
        error = ((X_batch - reconstructed) ** 2).mean(dim=1)  # Calcola l'errore per ogni settimana
        reconstruction_errors.extend(error.tolist())

In [16]:
# 6. Identifica le settimane anomale
threshold = np.percentile(reconstruction_errors, 95)  # Threshold al 95° percentile

# Etichetta per ogni settimana: 1 se anomala, 0 se normale
labels = [1 if error > threshold else 0 for error in reconstruction_errors]

In [17]:
# DataFrame per la visualizzazione
df_plot = pd.DataFrame({
    'Settimana': np.arange(1, len(reconstruction_errors) + 1),  # 60 settimane
    'Errore': reconstruction_errors,
    'Anomalia': labels
})

# Assicurati che la colonna 'Anomalia' sia di tipo categorico per un mapping discreto
df_plot['Anomalia'] = df_plot['Anomalia'].astype('category')

# Creazione del grafico con colori rosso e blu per le anomalie
fig = px.scatter(df_plot, x='Settimana', y='Errore', color='Anomalia',
                 color_discrete_map={0: 'blue', 1: 'red'},  # Associa il blu per 0 (normale) e rosso per 1 (anomalia)
                 title="Anomalie nelle settimane di sonno",
                 labels={'Errore': 'Errore di Ricostruzione', 'Settimana': 'Settimana'})

fig.show()